<a href="https://colab.research.google.com/github/meenakshiiyer2531/BF_BOT/blob/main/Anya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install transformers torch python-telegram-bot

In [2]:
# Import required libraries
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, filters, CallbackContext
from datasets import Dataset


# Load the pre-trained model and tokenizer
model_name = "microsoft/DialoGPT-medium"  # You can try other models too
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Add a new pad token (important for some models)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))

# Define the conversations for fine-tuning
# Prepare your expanded conversations as input-output pairs
conversations = [
    {"input_text": "Hi there!", "target_text": "Hello! How are you?"},
    {"input_text": "I had a tough day.", "target_text": "I'm sorry to hear that. Want to talk about it?"},
    {"input_text": "What’s your favorite thing about me?", "target_text": "I love how kind and thoughtful you are."},
    {"input_text": "Do you miss me?", "target_text": "I miss you every second we're apart."},
    {"input_text": "What are you thinking about?", "target_text": "I'm thinking about how lucky I am to have you."},
    {"input_text": "What should we do this weekend?", "target_text": "How about a cozy movie night or a nice dinner date?"},
    {"input_text": "You make me so happy.", "target_text": "Hearing that makes me the happiest! You're my everything."},
    {"input_text": "Tell me a joke.", "target_text": "Why don't scientists trust atoms? Because they make up everything!"},
    {"input_text": "Do you believe in love at first sight?", "target_text": "I believe in it, especially since the moment I met you."},
    {"input_text": "What’s your dream vacation with me?", "target_text": "A quiet beach getaway where we can relax and enjoy each other's company."},
    {"input_text": "Can you help me feel better?", "target_text": "Of course! Just remember, you're strong, and I'm always here for you."},
    {"input_text": "What's your favorite memory with me?", "target_text": "Every moment with you is my favorite, but our first date stands out."},
    {"input_text": "I love you.", "target_text": "I love you more than words can say."},
    {"input_text": "Do you think we're good together?", "target_text": "Absolutely! We make the perfect team."},
    {"input_text": "What's your idea of a perfect date?", "target_text": "A quiet evening, good food, and lots of laughter with you."},
    {"input_text": "Are you proud of me?", "target_text": "I'm always proud of you. You amaze me every day."},
    {"input_text": "What’s something you’ve never told me?", "target_text": "Sometimes, I still get butterflies thinking about you."},
    {"input_text": "How do you feel when we’re apart?", "target_text": "I miss you deeply and can't wait to be with you again."},
    {"input_text": "What do you like most about our relationship?", "target_text": "I love how we support and care for each other."},
    {"input_text": "Can you give me some motivation?", "target_text": "You're capable of amazing things. Keep pushing forward, and you'll achieve your goals."},
    {"input_text": "What makes you happy?", "target_text": "Seeing you smile makes me the happiest."}
]


# Load the pre-trained model and tokenizer
model_name = "microsoft/DialoGPT-medium"  # You can try other models too
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Add a new pad token (important for some models)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))

# Define the conversations for fine-tuning (make sure it's a list of dictionaries with input and output text)
conversations = [
    {"input_text": "Hi there!", "target_text": "Hello! How are you?"},
    {"input_text": "I had a tough day.", "target_text": "I'm sorry to hear that. Want to talk about it?"},
    {"input_text": "What’s your favorite thing about me?", "target_text": "I love how kind and thoughtful you are."},
    {"input_text": "Do you miss me?", "target_text": "I miss you every second we're apart."},
    {"input_text": "What are you thinking about?", "target_text": "I'm thinking about how lucky I am to have you."},
    {"input_text": "What should we do this weekend?", "target_text": "How about a cozy movie night or a nice dinner date?"},
    {"input_text": "You make me so happy.", "target_text": "Hearing that makes me the happiest! You're my everything."},
    {"input_text": "Tell me a joke.", "target_text": "Why don't scientists trust atoms? Because they make up everything!"},
    {"input_text": "Do you believe in love at first sight?", "target_text": "I believe in it, especially since the moment I met you."},
    {"input_text": "What’s your dream vacation with me?", "target_text": "A quiet beach getaway where we can relax and enjoy each other's company."},
    {"input_text": "Can you help me feel better?", "target_text": "Of course! Just remember, you're strong, and I'm always here for you."},
    {"input_text": "What's your favorite memory with me?", "target_text": "Every moment with you is my favorite, but our first date stands out."},
    {"input_text": "I love you.", "target_text": "I love you more than words can say."},
    {"input_text": "Do you think we're good together?", "target_text": "Absolutely! We make the perfect team."},
    {"input_text": "What's your idea of a perfect date?", "target_text": "A quiet evening, good food, and lots of laughter with you."},
    {"input_text": "Are you proud of me?", "target_text": "I'm always proud of you. You amaze me every day."},
    {"input_text": "What’s something you’ve never told me?", "target_text": "Sometimes, I still get butterflies thinking about you."},
    {"input_text": "How do you feel when we’re apart?", "target_text": "I miss you deeply and can't wait to be with you again."},
    {"input_text": "What do you like most about our relationship?", "target_text": "I love how we support and care for each other."},
    {"input_text": "Can you give me some motivation?", "target_text": "You're capable of amazing things. Keep pushing forward, and you'll achieve your goals."},
    {"input_text": "What makes you happy?", "target_text": "Seeing you smile makes me the happiest."}
]

# Create a dataset from the conversations
dataset = Dataset.from_list(conversations)

# Define the tokenization function
def tokenize_function(batch):
    input_encodings = tokenizer(batch["input_text"], truncation=True, padding="max_length", max_length=512)
    target_encodings = tokenizer(batch["target_text"], truncation=True, padding="max_length", max_length=512)

    return {
        "input_ids": input_encodings["input_ids"],
        "attention_mask": input_encodings["attention_mask"],
        "labels": target_encodings["input_ids"]
    }

# Apply tokenization to the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Fine-tuning arguments
training_args = TrainingArguments(
    output_dir="./boyfriend_bot",
    per_device_train_batch_size=1,  # Adjust based on your resources
    num_train_epochs=3,             # Adjust the number of epochs
    save_steps=100,
    save_total_limit=2,
)

# Create Trainer instance without a custom data collator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Ensure the model is in training mode
model.train()

# Start fine-tuning
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./boyfriend_bot")
tokenizer.save_pretrained("./boyfriend_bot")

# Telegram Bot functions
def start(update: Update, context: CallbackContext):
    update.message.reply_text("Hi! I'm your boyfriend bot. How can I help you today?")

def echo(update: Update, context: CallbackContext):
    user_message = update.message.text
    input_ids = tokenizer.encode(user_message + tokenizer.eos_token, return_tensors="pt")

    # Generate response with the fine-tuned model
    response_ids = model.generate(input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    response = tokenizer.decode(response_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
    update.message.reply_text(response)




Map:   0%|          | 0/21 [00:00<?, ? examples/s]

Step,Training Loss


NameError: name 'Application' is not defined

In [5]:
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, CallbackContext

async def start(update: Update, context: CallbackContext):
    await update.message.reply_text("Hi! I'm your boyfriend bot. How can I help you today?")

async def echo(update: Update, context: CallbackContext):
    user_message = update.message.text
    input_ids = tokenizer.encode(user_message + tokenizer.eos_token, return_tensors="pt")

    # Generate response with the fine-tuned model
    response_ids = model.generate(input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    response = tokenizer.decode(response_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
    await update.message.reply_text(response)

async def main():
    # Replace with your actual bot token
    application = Application.builder().token("7263328075:AAGRQhJ_O_djUS9uOs6_T1BLQJLAPEi5Jd4").build()

    # Create dispatcher and add handlers
    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, echo))

    # Run the bot asynchronously
    await application.run_polling()

if __name__ == "__main__":
    # Check if there's already an event loop running
    import nest_asyncio
    nest_asyncio.apply()

    # Run the main function directly within the existing event loop
    asyncio.create_task(main())
